In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv",header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [4]:
def replace(orig, percentage=0.1):
    """
    Replaces 'percentage'% of the original values in 'orig' with 99's
    :param orig: original data array
    :param percentage: percentage of values to replace (0<percentage<1)
    """
    new_data = orig.copy()
    rated = np.where(new_data!=98)
    n_rated = len(rated[0])
    idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
    new_data[rated[0][idx], rated[1][idx]] = 98
    return new_data, (rated[0][idx], rated[1][idx])


In [5]:
new_df,idx=replace(np.array(df))

In [6]:
len(new_df[0])

101

In [7]:
new_df.shape

(24983, 101)

 # Latent


In [9]:
from tqdm import tqdm
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=30):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in tqdm(range(iterations)):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
#                 if not np.isnan(rating):
                if rating!=99 and rating!=98:

                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

            
n_latent_factors = 10

user_ratings = new_df
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd()  # Note how the MSE decreases with the number of iterations



  3%|▎         | 1/30 [00:16<07:59, 16.55s/it]

39.14340963867811



  7%|▋         | 2/30 [00:32<07:41, 16.47s/it]

30.658938622329714



 10%|█         | 3/30 [00:50<07:34, 16.82s/it]

...

18.10388447242912



 97%|█████████▋| 29/30 [08:01<00:16, 16.56s/it]

18.04528348613226



100%|██████████| 30/30 [08:16<00:00, 16.22s/it]

18.000052111762418


In [11]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[77.732, -2.2  , -3.476, ..., -2.821, -3.631, -2.897],
       [95.259,  2.796,  2.384, ...,  2.881,  2.094,  3.707],
       [49.07 ,  6.095,  5.99 , ...,  6.071,  7.017,  6.861],
       ...,
       [39.18 , -0.705, -1.2  , ..., -0.986, -1.422, -1.171],
       [37.803,  0.937,  0.365, ...,  0.517,  0.573,  0.487],
       [72.741,  3.155,  3.061, ...,  3.119,  2.235,  3.571]])

In [12]:
new_df

array([[ 74.  ,  -7.82,  98.  , ...,  99.  ,  99.  ,  99.  ],
       [100.  ,   4.08,  -0.29, ...,   0.34,  -4.32,   1.07],
       [ 49.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  98.  ],
       ...,
       [ 39.  ,  99.  ,  98.  , ...,  99.  ,  98.  ,  99.  ],
       [ 37.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 72.  ,   2.43,   2.67, ...,  99.  ,  99.  ,  99.  ]])

In [13]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 77.73249119032646)","(-7.82, -2.2003562968877066)","(98.0, -3.4763301234829025)","(-9.66, -3.4995770038599434)","(98.0, -5.437923583400403)","(-7.52, -2.680114465351517)","(-8.5, -3.191305010522541)","(-9.85, -3.242064104652355)","(4.17, -2.3113432966325194)","(-8.98, -4.6524343692653325)",...,"(2.82, -1.1568487882400573)","(99.0, -2.39333736772124)","(99.0, -1.0662831094588785)","(99.0, -2.1331236541449785)","(99.0, -2.1185372347645037)","(99.0, -1.9727247325951338)","(-5.63, -1.644426079898582)","(99.0, -2.8205352336611815)","(99.0, -3.630661684184033)","(99.0, -2.8973984182779216)"
1,"(100.0, 95.25867148648008)","(4.08, 2.795830448342438)","(-0.29, 2.3842195814429448)","(6.36, 2.5732170674263846)","(4.37, 0.784138331224059)","(-2.38, 2.033462307658052)","(98.0, 4.020185948140941)","(-0.73, 1.2064483326484383)","(-5.34, 0.03409661013504778)","(8.88, 1.2264203202714508)",...,"(2.82, 3.668119047112903)","(-4.95, 2.9747712654960075)","(-0.29, 4.217863975977386)","(7.86, 2.8615549344578075)","(-0.19, 2.7066464180341234)","(-2.14, 3.232611091559263)","(3.06, 3.498185341059234)","(0.34, 2.881287184234904)","(-4.32, 2.0935988546553164)","(1.07, 3.7072239387351433)"
2,"(49.0, 49.07047589415346)","(99.0, 6.094529209490816)","(99.0, 5.989762418612234)","(99.0, 6.458178574317758)","(99.0, 5.915900931854488)","(9.03, 4.828816496886326)","(9.27, 7.45730946846422)","(98.0, 3.9100892557729647)","(9.27, 2.3051176591177804)","(99.0, 6.510951015716534)",...,"(99.0, 6.549117378612365)","(99.0, 6.610485238428407)","(98.0, 6.622465640000256)","(9.08, 6.65366383263536)","(99.0, 6.2555035538309305)","(99.0, 6.6772196009129985)","(99.0, 6.335359134121257)","(99.0, 6.071311198818799)","(99.0, 7.01652749120577)","(98.0, 6.861161281543396)"
3,"(48.0, 48.808963782791245)","(98.0, 2.149563652017324)","(8.35, 1.9755800604381515)","(99.0, 1.9901326838573572)","(99.0, 1.3471908701834285)","(1.8, 1.6935118802353617)","(8.16, 2.703225465895414)","(-2.82, 1.049719621571225)","(6.21, 0.5325698154682889)","(99.0, 1.742984085159761)",...,"(99.0, 2.711704071897446)","(99.0, 2.094645784567577)","(99.0, 2.9475708654073847)","(98.0, 2.1334730065827037)","(99.0, 1.922187952972819)","(99.0, 2.2749520201082)","(99.0, 2.539932301041792)","(99.0, 2.14821430673656)","(99.0, 1.4689621200961462)","(99.0, 2.391928909717246)"
4,"(91.0, 91.52322296491)","(8.5, 3.077491719580323)","(4.61, 2.717716532651879)","(-4.17, 2.967529126070448)","(-5.39, 1.1137299447334905)","(1.36, 2.639608401275288)","(1.6, 4.476926577658263)","(7.04, 1.6723388269864154)","(4.61, 0.718571966588096)","(-0.44, 2.131170395278333)",...,"(5.19, 4.270629899565159)","(98.0, 3.346008892824744)","(98.0, 5.014407186904023)","(5.19, 3.8121827958466175)","(98.0, 3.032346187669396)","(1.55, 3.6149242716029946)","(3.11, 4.339886366089392)","(6.55, 3.242324387436704)","(1.8, 2.73572791231696)","(1.6, 3.9186153664514736)"
5,"(100.0, 97.91187343576563)","(-6.17, -3.8157970839076953)","(-3.54, -5.359352385475919)","(0.44, -5.327812680858196)","(-8.5, -8.163546484786368)","(-7.09, -4.148685562552504)","(-4.32, -5.212132037721905)","(-8.69, -5.0224866679320925)","(-0.87, -3.2322278189893785)","(-6.65, -6.973293935280469)",...,"(-3.54, -2.4261047922511416)","(-6.89, -4.021758079292163)","(-0.68, -2.1428455535190696)","(-2.96, -3.321854683677306)","(-2.18, -3.4835276699670694)","(-3.35, -3.5118899379408317)","(0.05, -2.9624077731108747)","(-9.08, -4.622725869053458)","(-5.05, -5.569511772499222)","(-3.45, -4.877793202926153)"
6,"(47.0, 47.920422862362145)","(99.0, 3.2959391611568503)","(99.0, 3.416339359867946)","(99.0, 3.4034902591022393)","(99.0, 2.7026083627964295)","(8.59, 2.846014387346085)","(98.0, 4.825666149266035)","(7.72, 2.4784899022038824)","(8.79, 1.0138668682682088)","(99.0, 3.0731471910332666)",...,"(99.0, 3.509092144140612)","(99.0, 3.5130452141791886)","(99.0, 4.0927468940347325)","(98.0, 3.5169792002836706)","(99.0, 3.117398277496767)","(2.

In [23]:
mse=0
for i in range(predictions.shape[0]):
    values=user_ratings[i], predictions[i]
#     print(i ,">>>",values)
    for  j in range(predictions.shape[1]):
#         print(user_ratings[i][j], predictions[i][j])
        
        rating=user_ratings[i][j]
        
        if rating!=99 and rating!=98:
            mse+=(user_ratings[i][j]- predictions[i][j])**2

print("MSE ", mse/(predictions.shape[0]*predictions.shape[1]))
# comparison_data = pd.DataFrame(values)
# comparison_data

MSE  11.737927738748125
